In [1]:
 import findspark

In [3]:
findspark.init('/home/vboxuser/spark-3.5.0-bin-hadoop3')

In [4]:
import pyspark

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [13]:
dataset =  spark.read.csv('/home/vboxuser/Downloads/hack_data.csv' ,header=True, inferSchema = True)

In [14]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [15]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [16]:
from pyspark.ml.clustering import KMeans

In [17]:
feat_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
assesmbler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [22]:
final_data = assesmbler.transform(dataset)

In [23]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [24]:
from pyspark.ml.feature import StandardScaler

In [25]:
scaler = StandardScaler(inputCol = 'features', outputCol='scaledfeatures')

In [27]:
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [28]:
cluster_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledfeatures: vector (nullable = true)



In [32]:
kmeans2 = KMeans(featuresCol='scaledfeatures',k=2)
kmeans3 = KMeans(featuresCol='scaledfeatures',k=3)

In [33]:
model_k2=kmeans2.fit(cluster_final_data)
model_k3=kmeans3.fit(cluster_final_data)

24/01/11 11:53:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [36]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [37]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

